In [1]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from data_preprocessing import Autodata
from datasets import concatenate_datasets
from sklearn.model_selection import train_test_split
from dataloader import CustomDataset
from utils import compute_metrics

/Users/sangwon/.pyenv/versions/3.10.13/envs/cfn-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "monologg/koelectra-small-v3-discriminator"
data = Autodata("./data")
cfn_question = data.concat_dataset["question"]
normal_instruction_question = data.load_instruction_dataset("nlpai-lab/kullm-v2")

In [3]:
normal_instruction_question = normal_instruction_question[:1000]

In [4]:
cfn_data = data.label_indexing(cfn_question, state=0)
normal_instruction_data = data.label_indexing(normal_instruction_question, state=1)

total_data = concatenate_datasets([cfn_data, normal_instruction_data])
# train_dataset, val_dataset = train_test_split(
#     total_data, test_size=0.2, random_state=42
# )
question = total_data["question"]
target = total_data["target"]

In [5]:
question_train, question_valid, target_train, target_valid = train_test_split(
    question, target, test_size=0.2, random_state=42, stratify=target
)

In [7]:
train_data = CustomDataset(
    data=question_train,
    target=target_train,
    model_name=model_name,
    text_columns="question",
    max_length=256,
    state="train",
)

val_data = CustomDataset(
    data=question_valid,
    target=target_valid,
    model_name=model_name,
    text_columns="question",
    max_length=256,
    state="train",
)

Tokenizing: 100%|██████████| 333/333 [00:00<00:00, 10460.94it/s]


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, ignore_mismatched_sizes=True
)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
args = TrainingArguments(
    output_dir="output_dir",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    dataloader_num_workers=4,
    logging_steps=50,
    seed=42,
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/sangwon/.pyenv/versions/3.10.13/envs/cfn-llm/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  0%|          | 0/210 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already bee

{'eval_loss': 0.6844255328178406, 'eval_f1': 0.7504690431519699, 'eval_runtime': 31.6654, 'eval_samples_per_second': 10.516, 'eval_steps_per_second': 0.189, 'epoch': 1.0}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'eval_loss': 0.6841882467269897, 'eval_f1': 0.7504690431519699, 'eval_runtime': 29.3795, 'eval_samples_per_second': 11.334, 'eval_steps_per_second': 0.204, 'epoch': 2.0}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

100%|██████████| 210/210 [13:39<00:00,  7.20s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

{'eval_loss': 0.5021293759346008, 'eval_f1': 0.8980716253443526, 'eval_runtime': 29.4923, 'eval_samples_per_second': 11.291, 'eval_steps_per_second': 0.203, 'epoch': 10.0}


100%|██████████| 210/210 [14:09<00:00,  4.05s/it]

{'train_runtime': 849.6029, 'train_samples_per_second': 15.654, 'train_steps_per_second': 0.247, 'train_loss': 0.526094118754069, 'epoch': 10.0}


TrainOutput(global_step=210, training_loss=0.526094118754069, metrics={'train_runtime': 849.6029, 'train_samples_per_second': 15.654, 'train_steps_per_second': 0.247, 'train_loss': 0.526094118754069, 'epoch': 10.0})

In [11]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li